---
# Primeiro Passo - configuração do ambiente
---

Tutorial: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/get-set-up-for-amazon-ec2.html

## Criação de um usuário IAM (Instance and Access Management)

A Amazon não recomenda usar o login principal para esta finalidade.

* Acessar: https://console.aws.amazon.com/iam/;
* Users -> Add User;
* Escolher username;
* Marcar a caixa "AWS Management Console access";
* Definir senha;
* Criar grupo e adicionar a adicionar a política: "AdministratorAccess", encontrada pela filtragem (ou adicionar ao grupo **rnpi**);
* Finalizar criação;
* Página de login será: https://your_aws_account_id.signin.aws.amazon.com/console/, onde your_aws_account_id é o ID da conta AWS.

## Criar um Key Pair

Necessário para acessar a imagem mais tarde.

É necessário ter o arquivo na máquina que fará o acesso e o único momento em que você consegue baixar este arquivo é na hora da criação do mesmo.

* Key Pairs -> Create Key Pair;
* Definir um nome para o mesmo;
* Baixar o arquivo .pem;
* No terminal digitar: ``chmod 400 <your_user_name-key-pair-region_name>.pem`` para que somente o seu usuário tenha acesso.

## Utilizar VPC padrão

Não sei bem o que faz, mas não parece ser necessário mexer nisso.

## Criar Security Group

Pelo que entendi é necessário liberar acesso SSH para cada IP que vá tentar conectar-se à instância.

Além disso adicionam-se algumas portas padrão e também adicionei a porta para acesso do Jupyter (que rodará através do Docker)

* Security Groups -> Add Security Groups;
* Escolher um nome para o mesmo;
* Selecionar **Add Rule** e adicionar as seguintes regras:
    * Type: HTTP, Source: Anywhere (0.0.0.0/0)
    * Type: HTTPS, Source: Anywhere (0.0.0.0/0)
    * Type: SSH, Source: My IP (...) - *não recomendado liberar para todos os IPs*
    * Type: Custom TCP Rule, Port: ``<porta_jupyter>``, Source: Anywhere - *adicionado por mim*

---
# Segundo Passo - criação de uma instância
---

Tutorial: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html

* Acessar: https://console.aws.amazon.com/ec2/
* Selecionar **Launch Instance**
* Escolher a AMI (Amazon Machine Instance) - usei a padrão da Amazon
* Escolher o tipo de instância (g2.2xlarge)
* Review and Launch
* Selecionar security group criado
* Selecionar key pair criado
* **Launch Instance** - vai inicializar a instância

Para acessar a instância:

* Acessar: https://console.aws.amazon.com/ec2/
* Com o botão direito sobre a instância criada, selecionar **Connect** - não funciona no Chrome devido à remoção de algum suporte ao Java. Rodei com o Firefox.
* No primeiro acesso é necessário configurar a chave .pem baixada e aceitar alguns termos
* Após a inicialização do MindTerm, vem a instalação do Docker.

---
# Terceiro Passo - instalação do Docker
---

Tutorial: http://docs.aws.amazon.com/AmazonECS/latest/developerguide/docker-basics.html

No MindTerm digitar as seguintes linhas:

* ``sudo yum update -y``
* ``sudo yum install -y docker``
* ``sudo service docker start``
* ``sudo usermod -a -G docker ec2-user``

Talvez seja necessário reiniciar o sistema para que as permissões sejam atualizadas:
* ``sudo reboot``
* ``docker info`` - mostrará se o docker está rodando

---
# Quarto Passo - rodando o Jupyter através do Docker
---

Tutorial: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/README.md

Docker do curso de Deep Learning do Udacity:

* ``docker run -p 8888:8888 --name tensorflow-udacity -it gcr.io/tensorflow/udacity-assignments:0.6.0``

Onde 8888 é a porta configurada pelo Jupyter dentro do Docker, e também a porta configurada previamente com o grupo de segurança, para ser acessada de fora da máquina.

---
# Quinto Passo - acesso ao Jupyter
---

O endereço a ser acessado é o endereço que está em Public DNS na linha da instância em execução, na página do console EC2.

No formato ``<address>:8888`` ou a devida porta utilizada